In [ ]:
import pandas as pd
from operator import add
from functools import reduce
from sklearn.metrics import classification_report
import os

In [ ]:
lang = 'fa'
eval_set = 'dev'
dir = f'./{lang}/{eval_set}'

In [ ]:
files = os.listdir(dir)
files = [f for f in files if f.endswith('.csv')]
files

In [ ]:
dfs = [pd.read_csv(os.path.join(dir,f))  for f in files]

In [ ]:
merged = pd.DataFrame()
for i in range(len(dfs)):
    col = f'preds{i}'
    merged[col] = dfs[i].predictions
    merged[col] = merged[col].apply(lambda x: x.split())

# merged.shape

In [ ]:
merged['true'] = dfs[0].true
merged['true'] = merged['true'].apply(lambda x: x.split())

In [ ]:
preds = [reduce(add, merged['preds'+str(i)].array) for i in range(len(dfs))]
true = reduce(add, merged['true'].array)

In [ ]:
df = pd.DataFrame(preds)
df = df.T

In [ ]:
from collections import Counter

def ensemble(x):
    
    votes = Counter(x.values.tolist())
#     votes = sorted(votes.items(), key=lambda x: x[1], reverse=True)
    mx = 0
    winner = 'O'
    for key, val in votes.items():
        if val>mx:
            winner = key
            mx = val
        
    return winner

df['ensemble'] = df.apply(lambda x: ensemble(x), axis=1)
df.shape

In [ ]:
print(classification_report(df['ensemble'], true, output_dict=True)['macro avg']['f1-score'])